## Validation - Lookup Tables

In this notebook, we grab the full dynamic range of phase angles observed in the gold sample of WISE observations and randomly sample this range (with a randomly sampled range in $T_{ss}$). We sample 10,000 points in ($\alpha$, $T_{ss}$) and generate both interpolated and calculated fluxes. We compare the ratio of the interpolated values to the calculated values to determine the maximum deviation.

In [1]:
# Repository locations
ATM_DATA_DIR = "/epyc/projects/atm/atm_data/"

from atm import __version__
print("ATM Version: {}".format(__version__))

ATM Version: 1.0


In [2]:
import os 
import sys
import numpy as np 
import pandas as pd
import sqlite3 as sql
import matplotlib.pyplot as plt

from atm.models import STM, FRM, NEATM
from atm.obs import WISE

# Additonal imports to make simulated data
from atm import Config

%matplotlib inline

In [3]:
DPI = 300
SAVE_DIR = "../plots/"
FORMAT = "png"

SAVE_FIGS = True

In [4]:
con = sql.connect(os.path.join(ATM_DATA_DIR, "paper1/sample.db"))

In [5]:
# Instantiate observatory and NEATM class for simulating data
obs = WISE()
model = NEATM(verbose=False)

In [6]:
# Load WISE quadrature lookup tables into memory 
model.loadLambdaTables(obs.filterQuadratureLambdas, verbose=False)

In [7]:
observations = pd.read_sql("""SELECT * FROM observations WHERE keep == 1""", con)

In [8]:
max_alpha = np.radians(observations["alpha_deg"].values).max()
min_alpha = np.radians(observations["alpha_deg"].values).min()
mean_alpha = np.radians(observations["alpha_deg"].values).mean()

In [9]:
print(min_alpha)
print(max_alpha)
print(mean_alpha)

0.17411055552045032
1.4910469972835196
0.43990741193784505


In [10]:
T_ss = np.random.uniform(low=100, high=1200, size=10000)
alpha = np.random.uniform(low=0.0, high=np.pi/2, size=10000)

In [11]:
max_ratio = []
min_ratio = []
mean_ratio = []
sigma = []

for table in obs.filterQuadratureLambdas:
    flux_calc = model.calcTotalFluxLambdaEmittedToObsMany(np.array([table for i in range(len(T_ss))]), T_ss, alpha, threads=40)
    flux_interp = model.interpTotalFluxLambdaEmittedToObs(table, T_ss, alpha)
    
    ratio = flux_interp/flux_calc
    
    max_ratio.append(np.max(ratio))
    min_ratio.append(np.min(ratio))
    mean_ratio.append(np.mean(ratio))
    sigma.append(np.std(ratio))

In [12]:
mean_ratio

[0.9999999840960542,
 0.9999999919454925,
 0.9999999952476232,
 0.9999999965908822,
 0.9999999966674401,
 0.999999944590632,
 1.000000002017576,
 1.0000000323110998,
 1.000000068345077]

In [13]:
max_ratio

[1.0000757831211657,
 1.0000379214528976,
 1.0000173589918349,
 1.0000093522097009,
 1.0000081922293615,
 1.0000304002181821,
 1.0000668154743655,
 1.0001355923448865,
 1.0002241474099238]